## 1. 사전학습모델 만들기

In [ ]:
# 코랩을 이용해서 사용하였습니다.
# 아래의 과정은 yolov5 튜토리얼과정중 일부입니다.
# 깃허브 링크: https://github.com/ultralytics/yolov5 , 튜토리얼 링크 : https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb

# 1) 먼저, yolo 설치가 필요합니다. ===============================================================
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
# train_data.zip 파일을 저는 '/content'(%pwd) 에 넣었습니다.
# trani_data.zip 파일의 구성 : images 와 labels 폴더를 만들어 뒀습니다. 각 폴더 안에는 train 과 val 폴더가 있고, 사진과 라벨이 들어가있습니다.
# 2) train_data upzip 해줍니다===============================================================
!unzip -q ../train_data.zip -d ../

In [ ]:
# /content의 yolov5 폴더안의 data폴더에 custom_data.yaml 파일을 넣어줍니다.
# custom_data의 내용을 다음과 같습니다.
# ---------------------------------------------------------------------------------------------------
# train: /content/train_data/images/train  # train images (relative to 'path') 50 images
# val: /content/train_data/images/val  # val images (relative to 'path') 10 images
# test:  # test images (optional)

# # Classes
# nc: 7  # number of classes
# names: ['CARPAL', 'LMCP', 'MMCP', 'TMCP', 'LPIP', 'MPIP', 'IP']  # class names
# ----------------------------------------------------------------------------------------------------
# 3) *** batch, epochs 조절하고, custom_data.yaml 으로 수정해서 폴더에 넣어주고 실행합니다. ==============================================================
!python train.py --img 800 --batch 3 --epochs 500 --data custom_data.yaml --weights yolov5s.pt --cache

# 3번 과정의 실행이 완료되면 맨 마지막에 Results saved to runs/train/exp 라고 저장된 경로가 뜹니다.
# yolov5 폴더에 경로를 따라 들어가시면 확인하실 수 있습니다.


In [ ]:
# +) 박스 예측, test폴더를 만들어서 지정해주셔도 되고, 파일만 넣어도 detect 됩니다. 튜토리얼 과정을 참고하여 주세요. 여기서 저는 이것을 활용하지 않았습니다. 
# !python detect.py --weights runs/train/exp/weights/last.pt --img 320 --conf 0.25 --source ../717.jpg

## 2. 객체 인식 및 추출하기.

In [ ]:
# 4) 사전학습된 모델을 가져와서 예측(객체인식 및 추출)합니다. ==============================================================

#!pip install -qr requirements.txt 에 대한 런타임 재시작이 필요하다고 떠서 런타임을 재시작해 주었습니다.
# torch.hub.load 는 깃허브 리포지토리 또는 로컬 디렉터리에서 모델을 로드합니다. # 자세한 설명은 설명 사이트(https://pytorch.org/docs/stable/hub.html)를 참고해 주세요. # https://runebook.dev/ko/docs/pytorch/hub 이 사이트에서도 설명이 잘 되어있습니다.
# 리포지토리 또는 로컬 디렉터리 입력후 custom 이 들어간 자리는 모델명입니다. 
# model = torch.hub.load('repo_or_dir', 'model')  # yolov5에서 모델 종류는 yolov5s or yolov5m, yolov5l, yolov5x, custom 이렇게 있습니다.

import torch
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/yolov5/runs/train/exp/weights/last.pt', force_reload=True) 


# custom 모델은 path에 파일(가중치파일) 경로를 적용하여 사전 훈련된 모델을 생성합니다. 
# 위(로컬)에서 train 과 val 데이터로 사전 훈련된 모델(즉, 모델 정의 및 사전 훈련 된 가중치)을 가져와서 사용하였습니다.

# 샘플 이미지를 모델에 넣어보고 예측값을 받아옵니다.
img = '/content/sample.jpg'  # 이미지 경로를 지정해줍니다.
results = model(img)  
# crops = results.crop(save=True)  - render() 전에 crop 사용하면 박스 쳐지기 전에 잘라서 저장됨!!!  < 저희가 모을 파일입니다ㅎ

# results.print() 모델에 적용된 결과값이 출력됩니다.
# image 1/1: 800x600 1 CARPAL, 1 LMCP, 1 MMCP, 1 TMCP, 1 LPIP, 1 MPIP, 1 IP
# 이미지 하나를 적용했고, 크기는 800*600 이며 인식된 객체의 라벨과 개수입니다.
# Speed: 12.7ms pre-process, 12.9ms inference, 1.4ms NMS per image at shape (1, 3, 640, 480) # 그 밖의 결과입니다.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# results.imgs # array of original images (as np array) passed to model for inference
# results.render()  # updates results.imgs with boxes and labels
# render()을 이용하여 라벨이 붙은 이미지결과파일을 불러와서 np.squeeze로 길이가 1인 축을 제거합니다. - results의 image shape가 (1, 3, 640, 480) 인것을 위(results.print())에서 확인할 수 있었습니다.

plt.imshow(np.squeeze(results.render()))
plt.show()

In [ ]:
# 박스쳐진부분 잘려서 저장됨. 경로설정 가능. 
# render()후라서 박스와 같이 저장됨.
crops = results.crop(save=False)

In [ ]:
# 박스쳐진 전체 이미지를 저장합니다.
save_img = np.squeeze(results.render())
import cv2
cv2.imwrite('/content/sample.jpg', save_img)

In [ ]:
# 결과를 pandas로 받아올 수 있습니다.
print(results.pandas().xyxy[0])
#          xmin        ymin       xmax      ymax  confidence  class    name
# 0  171.875000  450.312500  390.00000  630.9375    0.937988      0  CARPAL
# 1  230.625000  274.218750  287.50000  350.0000    0.911621      2    MMCP
# 2  421.250000  377.187500  481.25000  429.6875    0.902832      3    TMCP
# 3  220.468750  144.140625  269.53125  182.8125    0.895020      5    MPIP
# 4   67.500000  253.125000  108.43750  291.5625    0.890137      4    LPIP
# 5  124.765625  327.187500  185.62500  382.8125    0.873047      1    LMCP
# 6  487.187500  309.843750  532.81250  352.8125    0.850586      6      IP